In [1]:
import timm
from urllib.request import urlopen
from PIL import Image

/home/lelouch/miniconda3/envs/vit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

In [3]:
model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k', pretrained=True)
model = model.eval()

In [4]:
print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 512, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=512, out_features=1536, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=512, out_features=512, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identit

In [5]:
# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [7]:
import torch
output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

In [11]:
print(top5_probabilities)
print(top5_class_indices)
print(output)
print(top5_probabilities[0].tolist())

tensor([[52.7900, 10.1153,  7.2810,  4.4503,  1.5161]],
       grad_fn=<TopkBackward0>)
tensor([[967, 504, 968, 415, 960]])
tensor([[-1.0360e-01, -7.5831e-01, -1.3608e-01, -3.9874e-02, -5.7988e-01,
          4.6911e-01,  9.7656e-03, -7.1957e-01, -2.8360e-01, -7.7765e-02,
          5.2384e-01, -2.1930e-01,  3.1100e-01, -6.9803e-02, -1.5805e-01,
          9.5728e-02,  1.6710e-01, -2.4809e-01, -5.6220e-01, -1.0428e-01,
         -5.1519e-01,  2.6381e-01,  1.0474e+00,  7.2704e-01,  1.1095e-01,
         -2.9215e-01, -7.7121e-01, -7.2072e-01, -1.5735e+00, -1.3539e+00,
         -2.3654e-01,  4.7543e-01,  1.4705e-01,  7.0121e-01, -3.0874e-01,
         -3.6605e-01, -2.5866e-01, -9.5356e-02,  3.4971e-01,  4.6263e-01,
         -9.9917e-02,  1.0015e-03,  9.2637e-01,  4.9731e-01, -6.2342e-01,
          1.4665e-01, -7.3544e-02, -1.5473e-01, -6.1598e-01,  4.2367e-01,
         -6.5245e-01, -2.9789e-01, -8.3461e-01,  3.1077e-02,  1.4136e-01,
          5.1430e-01, -7.8582e-01, -1.7560e-01, -3.6081e-01, -